In [1]:
import os
import re
import pandas as pd
import numpy as np
import sqlite3

In [2]:
PATH_TO_DATA = "../data/clickhouse_github_issue"

In [3]:
all_datasets = []
for folder in os.listdir(PATH_TO_DATA):
    try:
        for file in os.listdir(os.path.join(PATH_TO_DATA, folder)):
            print(f'Dataset {folder}')
            all_datasets.append(pd.read_json(os.path.join(PATH_TO_DATA, folder, file), lines=True))
    except NotADirectoryError:
        continue

Dataset closed_issues_with_labels
Dataset closed_issues_data
Dataset open_issues
Dataset open_issues_with_labels
Dataset open_issues_data
Dataset closed_issues


In [4]:
# curl == url

In [5]:
closed_issues_data = all_datasets[1]
open_issues = all_datasets[3]
open_issues_data = all_datasets[4] 
closed_issues = all_datasets[0] 

In [6]:
closed_issues_data.loc[0]

p                [Since Clickhouse support multi volume, there ...
code                                                            []
author                                             BhavyaRajSharma
relative-time                                 2021-10-07T10:59:20Z
ctime                                                   1635347274
curl             https://github.com/ClickHouse/ClickHouse/issue...
rowid                             0d699fe11a6898c568f85ee55598063c
taskid                            cdc3e3e05b3b8da4d7c56443c3a5e6f2
Name: 0, dtype: object

In [7]:
closed_issues.loc[0]

title                                   Failed to get Kafka data
url            https://github.com/ClickHouse/ClickHouse/issue...
time                                        2021-11-09T07:02:27Z
openedby                                                 sunxq86
issueLabels                                           [question]
ctime                                                 1636555513
curl           https://github.com/ClickHouse/ClickHouse/issue...
rowid                           faf887e835ae0a60987e9e7d34c38fd6
taskid                          0fb853cd160c67d7d203b4c6498d417c
Name: 0, dtype: object

# Looking at the problems in data

In [8]:
# 20% -> perc


In [11]:
SPECIAL_SYMBOL_G = closed_issues_data.iloc[53].p[4].split('\n')[3][0]
SPECIAL_SYMBOL_PALKA = closed_issues_data.iloc[53].p[4].split('\n')[4][0]
SPECIAL_SYMBOL_REVERSE_G = closed_issues_data.iloc[53].p[4].split('\n')[9][0]

In [641]:
regex = re.compile(r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))")
# url_pattern = re.compile(r'')
num = re.compile(r"\d*\.?\d+")
nick_re = re.compile(r"(?:\@|https?\://)\S+")
# fp_pattern = re.compile(r"\/\/\/(\w|\W){1,255}\.[\w]{3,4}")
fp_pattern = re.compile(r"[https:]\/.*?\.[\w:]+")
issue_ref_pattern = re.compile(r"#\d+")

# @den-crane
def sub_nicknames(text):
    text = re.sub(nick_re, 'nickname', text)
    return text

def sub_num(text):
    text = re.sub(num, ' num ', text)
    return text

def sub_code(text, code):
    for snippet in code:
        for snippet_part in [i.strip() for i in snippet.split('/n') if len(i.strip()) > 0]:
            text = re.sub(re.escape(snippet_part), 'code', text)
    return text

def clean_paths(text):
    text = re.sub(fp_pattern, 'fpath', text)
    return text
    
def sub_issue_refs(text):
    text = re.sub(issue_ref_pattern, ' ', text)
    return text

def clean_repetitions(text):
    splitted_text = text.split()
    idx = [i for i in range(1, len(splitted_text)) if (splitted_text[i].lower() == splitted_text[i-1].lower())]
    for i in idx[::-1]:
        del splitted_text[i]
    idx = [i for i in range(2, len(splitted_text)) if (splitted_text[i].lower() == splitted_text[i-2].lower())]
    for i in idx[::-1]:
        del splitted_text[i]
    return ' '.join(splitted_text)

def link_search(text):
    url = re.findall(regex, text)  
    for link in url:
        try:
            text = re.sub(re.escape(link[0]), 'link', text)
        except:
            break
    return text

def remove_excuses(list_of_texts):
    text_to_merge = []
    for text in list_of_texts:
        if ('Sorry, something went wrong.' in text) or ('The text was updated successfully, but these errors were encountered:' in text):
            continue
        else:
            text_to_merge.append(text)
    result = '\n'.join(text_to_merge)
    return result
    
    
def check_code_injectons(list_of_texts):
    ok_texts = []
    for text in list_of_texts:
        if text[0] == SPECIAL_SYMBOL_G:
            continue
        elif text[0] == SPECIAL_SYMBOL_PALKA:
            continue
        elif text[0] == SPECIAL_SYMBOL_REVERSE_G:
            continue
        elif text.lower().strip().startswith('partition by'):
            continue
        elif text.lower().strip().startswith('order by'):
            continue
        elif text.lower().strip().startswith('engine ='):
            continue
        elif text.lower().strip().startswith('optimize table'):
            continue
        elif text.lower().strip().startswith('show databases'):
            continue
        elif text.lower().strip().startswith('->'):
            continue
        elif text.lower().strip().startswith('( ...'):
            continue
        elif text.lower().strip().startswith(')'):
            continue
        elif text.lower().strip().startswith('gnu'):
            continue
        elif text.lower().strip().startswith('#'):
            continue
        elif text.lower().strip().startswith('gdb'):
            continue
        elif text.lower().strip().startswith('copyright'):
            continue
        elif text.lower().strip().startswith('attaching to'):
            continue
        elif text.lower().strip().startswith('loaded symbols'):
            continue
        elif text.lower().strip().startswith('reading symbols'):
            continue
        elif text.lower().strip().startswith('['):
            continue
        elif text.lower().strip().startswith('thread'):
            continue
        elif text.lower().strip().startswith('pthread'):
            continue
        elif text.count(':') >= 2:
            continue
        elif text.count('|') >= 1:
            continue
        elif text.count('-') >= 2:
            continue
        elif text.count('*') >= 3:
            continue
        elif text.count('_') >= 2:
            continue
        elif text.lower().strip().startswith('no symbol'):
            continue   
        elif text.lower().strip().startswith('run build'):
            continue
        elif text.lower().strip().startswith('source file'):
            continue
        elif text.lower().strip().startswith('change dir'):
            continue
        elif text.lower().strip().startswith('performing c'):
            continue
        elif text.lower().strip().startswith('void'):
            continue
        elif text.lower().strip().startswith('int'):
            continue
        elif text.lower().strip().startswith('char'):
            continue
        elif text.lower().strip().startswith('return'):
            continue
        elif text.lower().strip().startswith('failed'):
            continue
        elif text.lower().strip().startswith('ninja'):
            continue
        elif text.lower().strip().startswith('cmakefiles'):
            continue
        elif text.lower().strip().startswith('checkincludefile'):
            continue
        elif text.lower().strip().startswith('^'):
            continue
        elif text.lower().strip().startswith('source'):
            continue
        elif text.lower().strip().startswith('collect'):
            continue
        elif text.lower().strip().startswith('static'):
            continue
        elif text.lower().strip().startswith('{'):
            continue
        elif text.lower().strip().startswith('}'):
            continue
        elif text.lower().strip().startswith('determining'):
            continue
        elif text.lower().strip().startswith('booked by'):
            continue
        elif text.lower().strip().startswith('done.'):
            continue
        elif text.lower().strip().startswith('the task'):
            continue
        elif text.lower().strip().startswith('intermediate stage'):
            continue
        elif text.lower().strip().startswith('determining'):
            continue
        elif text.lower().strip().startswith('review stage'):
            continue
        elif text.lower().strip().startswith('this topic is booked'):
            continue
        elif text.lower().strip().startswith('low percentage'):
            continue
        elif text.lower().strip().startswith('unknown status'):
            continue
        elif text.lower().strip().startswith('see also'):
            continue
        elif text.lower().strip().startswith('?'):
            continue
        elif text.lower().strip().startswith('in file included'):
            continue
        elif text.lower().strip().startswith('file'):
            continue
        elif text.lower().strip().startswith('from'):
            continue
        elif text.lower().strip().startswith('reading packag'):
            continue
        elif text.lower().strip().startswith('setting up'):
            continue
        elif text.lower().strip().startswith('clickHouse binary'):
            continue
        elif text.lower().strip().startswith('creating symlink'):
            continue
        elif text.lower().strip().startswith('start clickhouse-server'):
            continue
        elif text.lower().strip().startswith('start clickhouse-client'):
            continue
        elif text.lower().strip().startswith('synchronizing state'):
            continue
        elif text.lower().strip().startswith('preparing to unpack'):
            continue
        elif text.lower().strip().startswith('clickhouse binary'):
            continue
        elif text.lower().strip().startswith('creating user'):
            continue
        elif text.lower().strip().startswith('pid directory'):
            continue
        elif text.lower().strip().startswith('chown'):
            continue
        elif text.lower().strip().startswith('clickhouse has been'):
            continue
        elif text.lower().strip().startswith('processing triggers'):
            continue
        elif text.lower().strip().startswith('datetime'):
            continue
        elif text.lower().strip().startswith('host'):
            continue
        elif text.lower().strip().startswith('ptcl'):
            continue
        elif text.lower().strip().startswith('php'):
            continue
        elif text.lower().strip().startswith('pqsn'):
            continue
        elif 'uint64' in text.lower():
            continue
        elif 'fatal error' in text.lower():
            continue
        elif 'errors generated' in text.lower():
            continue
        elif 'found in' in text.lower():
            continue
        elif '-fstack' in text.lower():
            continue
        elif 'ignore line' in text.lower():
            continue
        elif '==>' in text.lower():
            continue
        elif '=>' in text.lower():
            continue
        elif 'unpacking' in text.lower():
            continue
        elif 'replace the old symlink' in text.lower():
            continue
        elif 'already exists but it points to' in text.lower():
            continue
        elif 'replace the old symlink' in text.lower():
            continue
        elif any(i in text.lower() for i in ['enum8', 'uint16', 'int64', 
                                             'uint32', 'tcp_swapfail_miss', 
                                             'http/1.0', 'linux release',
                                             'kvm virtual machine', 'to fix try run', 
                                             '- >', 'isnotnull', '<=', 'int8', 'int64',
                                             'datetime default', 'decimal', 'index_granularity',
                                             '+=', 'cmake version', 'compile error', 'error generated',
                                             'const char', 'alter partition'
                                            ]):
            continue
        ok_texts.append(text)
    return ok_texts

temp_db = sqlite3.connect(":memory:")

def sql_code_detector(list_of_snt):
    text_list = []
    for snt in list_of_snt:
        try:
            temp_db.execute(snt)
        except Exception as e:
            s = str(e)
            if 'syntax error' in s:
#                 continues
                if s.split()[1].split('"')[1] != snt.split()[0]:
                    continue
                text_list.append(snt)
    return ' '.join(text_list)
    
def clean_text(row):
#     print(row)
    list_of_texts = row.p
#     print(f'Original: {list_of_texts}')
    text = remove_excuses(list_of_texts)
    text_splitted = [i.strip() for i in text.split('\n') if len(i.strip()) > 0]
    text_splitted = check_code_injectons(text_splitted)
    text = sql_code_detector(text_splitted)
    text = sub_code(text, row.code)
    text = link_search(text)
    text = sub_num(text)
    text = clean_paths(text)
    text = sub_issue_refs(text)
    text = clean_repetitions(text)
    text = sub_nicknames(text)
#     print(f'Cleaned: {text}')
#     print()
    return text
    
def convert_list_to_str(list_data, sep=';'):
    return sep.join(list_data)
    

# def data_combining():

    #22236 - clear
    # /var/log/clickhouse-server/clickhouse-server.err.log

In [642]:
check_code_injectons(["SHOW DATABASES LIKE 'test'"])

[]

In [643]:
example_data = '\n'.join(closed_issues_data.iloc[6].p)
example_data = [i.strip() for i in example_data.split('\n') if len(i.strip()) > 0]
print(example_data)
sql_code_detector(example_data)

["I found that's because of the hadoop.rpc.protection=PRIVACY", '#22236', 'Sorry, something went wrong.']


"I found that's because of the hadoop.rpc.protection=PRIVACY #22236"

In [644]:
example = 'CH support JBOD, but it recommended RAID as per the link https://clickhouse.com/docs/en/operations/tips/#storage-subsystem . Any Reasons? we are evaluating if we can move away from RAID and use JBOD, hence this question.'
example = link_search(example)
clean_repetitions(example)

'CH support JBOD, but it recommended RAID as per the link . Any Reasons? we are evaluating if we can move away from RAID and use JBOD, hence this question.'

In [645]:
# title, comments, labels, url, /var/log/clickhouse-server/clickhouse-server.err.log

In [646]:
sample_data = closed_issues_data.head(105)
sample_data.apply(clean_text, axis=1)

0      Since Clickhouse support multi volume, there i...
1      Merge reads several source parts and writes to...
2      Merge reads several source parts and writes to...
3      CH support JBOD, but it recommended RAID as pe...
4      RAID is num - times more performant, because i...
                             ...                        
100                                 Always reproducible.
101                                        Line num in a
102                  nickname code is variadic function.
103    Lines num to in a So we need to check method g...
104    Describe the unexpected behaviour Materialized...
Length: 105, dtype: object

In [647]:
sample_data

,p,code,author,relative-time,ctime,curl,rowid,taskid,clear_text,len
0,"[Since Clickhouse support multi volume, there ...",[],BhavyaRajSharma,2021-10-07T10:59:20Z,1635347274,https://github.com/ClickHouse/ClickHouse/issue...,0d699fe11a6898c568f85ee55598063c,cdc3e3e05b3b8da4d7c56443c3a5e6f2,"Since Clickhouse support multi volume, there i...",48
1,[Merge reads several source parts and writes t...,[],den-crane,2021-10-07T12:23:28Z,1635347274,https://github.com/ClickHouse/ClickHouse/issue...,d886c592696f5be34cbe926214ffcefc,cdc3e3e05b3b8da4d7c56443c3a5e6f2,Merge reads several source parts and writes to...,34
2,[Merge reads several source parts and writes t...,[],den-crane,2021-10-07T12:23:28Z,1635347274,https://github.com/ClickHouse/ClickHouse/issue...,d886c592696f5be34cbe926214ffcefc,cdc3e3e05b3b8da4d7c56443c3a5e6f2,Merge reads several source parts and writes to...,34
3,"[CH support JBOD, but it recommended RAID as p...",[],BhavyaRajSharma,2021-10-07T10:54:09Z,1635347275,https://github.com/ClickHouse/ClickHouse/issue...,1996da6f70342d128cb3ae9eb0871694,a32a821618c8dc5d7495387c0dd3fd8f,"CH support JBOD, but it recommended RAID as pe...",30
4,"[RAID is 5-10 times more performant, because i...",[],den-crane,2021-10-07T12:20:12Z,1635347275,https://github.com/ClickHouse/ClickHouse/issue...,ec828f3c0a575262da4bfc10e35d600b,a32a821618c8dc5d7495387c0dd3fd8f,"RAID is num - times more performant, because i...",63
...,...,...,...,...,...,...,...,...,...,...
100,"[Always reproducible., \n \n \n \n\n , \...",[],alexey-milovidov,2021-10-22T04:17:26Z,1635347345,https://github.com/ClickHouse/ClickHouse/issue...,3ce940c7722d7dbdec6d48a18966de2b,eb2aa148b2a6560f01e31deb0d63d718,Always reproducible.,2
101,"[It's connected with short circuit, we cannot ...",[],Avogar,2021-10-22T05:13:18Z,1635347345,https://github.com/ClickHouse/ClickHouse/issue...,9a06b29621930653b9bd8cd0e033b74e,eb2aa148b2a6560f01e31deb0d63d718,Line num in a,4
102,"[@Avogar now64 is variadic function., \n \n...",[now64],alexey-milovidov,2021-10-22T19:06:09Z,1635347345,https://github.com/ClickHouse/ClickHouse/issue...,f5b110a7d3544d3990dc8dda317215c0,eb2aa148b2a6560f01e31deb0d63d718,nickname code is variadic function.,5
103,"[Oh, sorry, tried to find the problem from the...",[],Avogar,2021-10-23T05:48:34Z,1635347345,https://github.com/ClickHouse/ClickHouse/issue...,f6d757a3f2983c331f5877f9512b1242,eb2aa148b2a6560f01e31deb0d63d718,Lines num to in a So we need to check method g...,44


In [ ]:
# labels & get theta

In [648]:
sample_data.iloc[53].p[4].split('\n')

['detach part and check parts:',
 "alter table table1 detach part '20211003_1_5_1';",
 "select partition,name,part_type,active,bytes_on_disk from system.parts where table = 'table1';",
 '┌─partition──┬─name───────────┬─part_type─┬─active─┬─bytes_on_disk─┐',
 '│ 2021-10-03 │ 20211003_1_1_0 │ Wide │ 0 │ 4655368 │',
 '│ 2021-10-03 │ 20211003_2_2_0 │ Wide │ 0 │ 4655723 │',
 '│ 2021-10-03 │ 20211003_3_3_0 │ Wide │ 0 │ 4653946 │',
 '│ 2021-10-03 │ 20211003_4_4_0 │ Wide │ 0 │ 4654628 │',
 '│ 2021-10-03 │ 20211003_5_5_0 │ Wide │ 0 │ 4655027 │',
 '└────────────┴────────────────┴───────────┴────────┴───────────────┘']

In [649]:
closed_issues_data['clear_text'] = closed_issues_data.apply(clean_text, axis=1)
open_issues_data['clear_text'] = open_issues_data.apply(clean_text, axis=1)

In [650]:
def get_len(text):
    return len(text.split())

In [651]:
closed_issues_data['len'] = closed_issues_data['clear_text'].apply(get_len)

In [652]:
closed_issues_data.sort_values('len', ascending=False).head(30)

,p,code,author,relative-time,ctime,curl,rowid,taskid,clear_text,len
11379,[This is the list of proposed tasks. It is to ...,"[-fsanitize=coverage, -fxray-instrument, pg2ch...",alexey-milovidov,2020-09-20T23:45:08Z,1635352299,https://github.com/ClickHouse/ClickHouse/issue...,81c4ced2eb139cb87bdb957f7a97e8f4,bb25b499556b9d21a352acc9c8d4059f,This is the list of proposed tasks. It is to b...,661
22567,[We have some replicated tables for over a yea...,"[<Error> prod.KtHistoryItem: Code: 40, e.displ...",bessonov87,2019-08-12T18:12:29Z,1635358453,https://github.com/ClickHouse/ClickHouse/issue...,1be22eb29be731d9a8c99e577df3ef83,f012f15d7d77d3870add2265d3818a1e,We have some replicated tables for over a year...,464
28503,[Hello\nI'm starting to use this function but ...,[],TH-HA,2018-09-14T15:08:31Z,1635360508,https://github.com/ClickHouse/ClickHouse/issue...,88b3e2db8442cfa69119fb3503661fc1,efade29665a631804eecaa4ef3b70a91,Hello To describe very quickly the use case: W...,422
20900,[There are some features connected with machin...,[SELECT sqrt(avg(diff * diff)) AS MSE from ......,KochetovNicolai,2019-10-16T07:22:40Z,1635356127,https://github.com/ClickHouse/ClickHouse/issue...,33f451272abbd0a53e8766b41a7a9e70,0b4c34a21ac564e3df2cbe12ed9c17b2,There are some features connected with machine...,411
9849,[We have a set of data parts that is dynamical...,[ /** Minimum ratio of size of one part...,alexey-milovidov,2020-11-02T18:18:14Z,1635351165,https://github.com/ClickHouse/ClickHouse/issue...,90893cbd49bb3ed9b51f4e80400675f8,16cb6dd7db8894759e1a9a1ceead02f8,We have a set of data parts that is dynamicall...,409
27639,"[Hi, I have the following MariaDB table in my ...","[partition key, order by, Partition key, order...",healiseu,2018-11-12T22:13:21Z,1635360299,https://github.com/ClickHouse/ClickHouse/issue...,39a9102b1ccbc638937e1efdefb023d1,05243708336b340be7312085b1180c86,Is the following CREATE command a good candida...,392
21568,"[Sorry, my statement was not complete. Protobu...",[],LizardWizzard,2020-05-12T07:45:37Z,1635356341,https://github.com/ClickHouse/ClickHouse/issue...,bfd1d0b32d7967224321bfc91635d53a,dc6ad0c84c2d5e14419e5fafb3736fd9,without introducing situations like one you po...,388
30591,[To utilize AES-NI instruction the only algori...,"[if encrypt(A,KEY) = encrypt(B,KEY) then A=B\n...",filimonov,2018-03-19T14:47:56Z,1635361797,https://github.com/ClickHouse/ClickHouse/issue...,a6f37b8d9c5f9750ac2f69c580c2c4ea,7085f396a7e6f95b0de761c7448aa225,To utilize AES-NI instruction the only algorit...,384
28077,[@filimonov this is an update of a test I am c...,[TripFlat\n\n┌─pk─┬─duration─┬──────────start_...,healiseu,2018-10-17T14:51:28Z,1635360424,https://github.com/ClickHouse/ClickHouse/issue...,b1bacd45399f5fe5f58bd8ae683e03ff,6cd91a5c4b42a789bc475d9f80e5f6f1,nickname this is an update of a test I am curr...,357
28842,[cmake3 --version\ncmake3 version 3.6.1\ngcc -...,"[LLVMConfig.cmake\nllvm-config.cmake\n, LLVMCo...",qinzl1,2018-08-15T08:11:51Z,1635360582,https://github.com/ClickHouse/ClickHouse/issue...,67c632552efa588c2c98a7e5ce937b4f,9db9a6d8d1318002863aaa26f919ab43,cmake num version gcc (GCC) num (Red Hat num -...,356


In [639]:
closed_issues_data.sort_values('len', ascending=False).head(30).loc[27556].clear_text

"Hello I just want to share a few thougths on this subject as i 'm looking to implement the same thing. I can imagine num evolution to reach this goal : num ALTER PARTITION MOVE PART TO [NODE.]PATH Allow to move a partition to a different path . Of course move is safe : ie. Copy the file ( let the old one still active during the copy ) , an only when the copy is done and verified , update the reference. During this phase , block any merge , if there is new part , copy them also. PATH is not the physical path , but some reference from the configuration file ( hotpath , coldpath ...... ) . Of course this shall be done online with minimum lock possible. Optionally if the table is distributed, the partition can also be moved to another host... num : Put a cron for maintenance scripts in clickhouse. Allowing to run some 'sql script' on a DB and run them at regular interval. (Can be done externally but i like the idea of this being part of the schema to make things consistent together). num 

In [640]:
closed_issues_data.sort_values('len', ascending=False).head(30).loc[27556].p

["Hello\nI just want to share a few thougths on this subject as i 'm looking to implement the same thing.",
 'I can imagine 3 evolution to reach this goal :\n1 : ALTER PARTITION MOVE PART TO [NODE.]PATH\nAllow to move a partition to a different path . Of course move is safe : ie. Copy the file ( let the old one still active during the copy ) , an only when the copy is done and verified , update the reference. During this phase , block any merge , if there is new part , copy them also.\nPATH is not the physical path , but some reference from the configuration file ( hotpath , coldpath ...... ) .\nOf course this shall be done online with minimum lock possible. Optionally if the table is distributed, the partition can also be moved to another host...',
 "2 : Put a cron for maintenance scripts in clickhouse. Allowing to run some 'sql script' on a DB and run them at regular interval. (Can be done externally but i like the idea of this being part of the schema to make things consistent toget

In [653]:
closed_issues_data['type'] ='closed'
open_issues_data['type'] = 'open'

In [654]:
closed_issues = closed_issues[['title', 'url', 'issueLabels']]
open_issues = open_issues[['title', 'url', 'issueLabels']]

In [655]:
closed_issues['issueLabels'] = closed_issues['issueLabels'].apply(convert_list_to_str)
open_issues['issueLabels'] = open_issues['issueLabels'].apply(convert_list_to_str)

/home/shared/.local/lib/python3.8/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


In [656]:
closed_issues = closed_issues.drop_duplicates()
open_issues = open_issues.drop_duplicates()
# closed_issues_data = closed_issues_data.drop_duplicates()
# open_issues_data = open_issues_data.drop_duplicates()

In [657]:
closed_issues_data

,p,code,author,relative-time,ctime,curl,rowid,taskid,clear_text,len,type
0,"[Since Clickhouse support multi volume, there ...",[],BhavyaRajSharma,2021-10-07T10:59:20Z,1635347274,https://github.com/ClickHouse/ClickHouse/issue...,0d699fe11a6898c568f85ee55598063c,cdc3e3e05b3b8da4d7c56443c3a5e6f2,"Since Clickhouse support multi volume, there i...",48,closed
1,[Merge reads several source parts and writes t...,[],den-crane,2021-10-07T12:23:28Z,1635347274,https://github.com/ClickHouse/ClickHouse/issue...,d886c592696f5be34cbe926214ffcefc,cdc3e3e05b3b8da4d7c56443c3a5e6f2,Merge reads several source parts and writes to...,34,closed
2,[Merge reads several source parts and writes t...,[],den-crane,2021-10-07T12:23:28Z,1635347274,https://github.com/ClickHouse/ClickHouse/issue...,d886c592696f5be34cbe926214ffcefc,cdc3e3e05b3b8da4d7c56443c3a5e6f2,Merge reads several source parts and writes to...,34,closed
3,"[CH support JBOD, but it recommended RAID as p...",[],BhavyaRajSharma,2021-10-07T10:54:09Z,1635347275,https://github.com/ClickHouse/ClickHouse/issue...,1996da6f70342d128cb3ae9eb0871694,a32a821618c8dc5d7495387c0dd3fd8f,"CH support JBOD, but it recommended RAID as pe...",30,closed
4,"[RAID is 5-10 times more performant, because i...",[],den-crane,2021-10-07T12:20:12Z,1635347275,https://github.com/ClickHouse/ClickHouse/issue...,ec828f3c0a575262da4bfc10e35d600b,a32a821618c8dc5d7495387c0dd3fd8f,"RAID is num - times more performant, because i...",63,closed
...,...,...,...,...,...,...,...,...,...,...,...
34975,"[fixed in #26140, \n \n \n \n\n , \n ...",[],den-crane,2021-08-28T15:31:53Z,1635362952,https://github.com/ClickHouse/ClickHouse/issue...,589190f8915f073593ef6f54096cff92,b7a9dba45ded08642689f4837966ab82,fixed in # num,4,closed
34976,[can anyone implement sha512 function in Hash ...,[],SachinPatil4991,2021-08-18T07:50:31Z,1635362956,https://github.com/ClickHouse/ClickHouse/issue...,8f66f53b3736d78dd7df91d122fad2ae,e847b558085b3ff23b24d723c608254e,can anyone implement sha num function in Hash ...,9,closed
34977,"[#27830 @alexey-milovidov, \n \n \n \n\n ...",[],zhanglistar,2021-08-19T02:59:48Z,1635362956,https://github.com/ClickHouse/ClickHouse/issue...,5dc22d4a47e6d9843bdc1049e3c60a58,e847b558085b3ff23b24d723c608254e,,0,closed
34978,"[Hello,, I'm going to use the timezone., I che...",[],parkbaesun,2021-08-05T09:21:58Z,1635362960,https://github.com/ClickHouse/ClickHouse/issue...,f9b158971e6752443d1171ffb9469cb9,31a4a327ee6de13848f6bd09ec9bf678,"I checked the contents in the document, but I ...",45,closed


In [658]:
# TODO: figure out what to do with the code

In [659]:
closed_issues_data['clear_text'] = closed_issues_data.groupby(['curl'])['clear_text'].transform(lambda x: ' '.join(x))
closed_issues_data = closed_issues_data[['author', 'curl', 'clear_text', 'type']].drop_duplicates()

In [660]:
open_issues_data['clear_text'] = open_issues_data.groupby(['curl'])['clear_text'].transform(lambda x: ' '.join(x))
open_issues_data = open_issues_data[['author', 'curl', 'clear_text', 'type']].drop_duplicates()

In [661]:
open_issues_data.head()

,author,curl,clear_text,type
0,den-crane,https://github.com/ClickHouse/ClickHouse/issue...,no issue in case of code,open
1,vegastar002,https://github.com/ClickHouse/ClickHouse/issue...,clickhouse version : num mysql version : num I...,open
2,matwey,https://github.com/ClickHouse/ClickHouse/issue...,Operating system openSUSE Tumbleweed glibc num...,open
3,UnamedRus,https://github.com/ClickHouse/ClickHouse/issue...,Describe the unexpected behaviour If you will ...,open
4,UnamedRus,https://github.com/ClickHouse/ClickHouse/issue...,Describe the unexpected behaviour toStartOf fu...,open


In [662]:
# merging together
df_closed = pd.merge(closed_issues_data, closed_issues, left_on='curl', right_on='url')
df_opened = pd.merge(open_issues_data, open_issues, left_on='curl', right_on='url')

In [663]:
df_closed.head()

,author,curl,clear_text,type,title,url,issueLabels
0,BhavyaRajSharma,https://github.com/ClickHouse/ClickHouse/issue...,"Since Clickhouse support multi volume, there i...",closed,PARTS ON DIFFERENT DISK - MERGE OPERATION - MU...,https://github.com/ClickHouse/ClickHouse/issue...,question;question-answered
1,den-crane,https://github.com/ClickHouse/ClickHouse/issue...,"Since Clickhouse support multi volume, there i...",closed,PARTS ON DIFFERENT DISK - MERGE OPERATION - MU...,https://github.com/ClickHouse/ClickHouse/issue...,question;question-answered
2,BhavyaRajSharma,https://github.com/ClickHouse/ClickHouse/issue...,"CH support JBOD, but it recommended RAID as pe...",closed,WHY RAID RECOMMENDED?,https://github.com/ClickHouse/ClickHouse/issue...,question;question-answered
3,den-crane,https://github.com/ClickHouse/ClickHouse/issue...,"CH support JBOD, but it recommended RAID as pe...",closed,WHY RAID RECOMMENDED?,https://github.com/ClickHouse/ClickHouse/issue...,question;question-answered
4,lala7573,https://github.com/ClickHouse/ClickHouse/issue...,I have trouble with connecting code. when I tr...,closed,clickhouse hdfs works strange..,https://github.com/ClickHouse/ClickHouse/issue...,question


In [665]:
df_opened.head()

,author,curl,clear_text,type,title,url,issueLabels
0,den-crane,https://github.com/ClickHouse/ClickHouse/issue...,no issue in case of code,open,"ASOF JOIN LowCardinality and constant, Expecte...",https://github.com/ClickHouse/ClickHouse/issue...,unexpected behaviour
1,vegastar002,https://github.com/ClickHouse/ClickHouse/issue...,clickhouse version : num mysql version : num I...,open,about MaterializeMySQL change PARTITION BY toY...,https://github.com/ClickHouse/ClickHouse/issue...,question
2,matwey,https://github.com/ClickHouse/ClickHouse/issue...,Operating system openSUSE Tumbleweed glibc num...,open,__pthread_mutex_(lock|unlock) are not exported...,https://github.com/ClickHouse/ClickHouse/issue...,build
3,UnamedRus,https://github.com/ClickHouse/ClickHouse/issue...,Describe the unexpected behaviour If you will ...,open,date column + WITH FILL timestamp return meani...,https://github.com/ClickHouse/ClickHouse/issue...,unexpected behaviour
4,UnamedRus,https://github.com/ClickHouse/ClickHouse/issue...,Describe the unexpected behaviour toStartOf fu...,open,"toStartOfInterval(now(),INTERVAL 1 WEEK) and t...",https://github.com/ClickHouse/ClickHouse/issue...,unexpected behaviour


In [666]:
all_df = pd.concat([df_closed, df_opened])

In [667]:
all_df['clear_text'] = all_df['title'] + ' ' + all_df['clear_text']

In [ ]:
# all_df['clear_text'] = all_df['clear_text'].apply(clear_text)

In [ ]:
len(all_df['clear_text'].unique())

In [668]:
all_df = all_df[['clear_text', 'type', 'issueLabels', 'title', 'curl']].drop_duplicates().reset_index()

In [ ]:
all_df['curl'][1]

In [ ]:
all_df.head()

In [669]:
all_df.to_csv(os.path.join(PATH_TO_DATA, 'all_issues_cleaned_v2.csv'), index=None)

In [670]:
# ! rm -r /mnt/ess_storage/DN_1/storage/home/khodorchenko/GOTM/datasets/clickhouse_issues
# ! mkdir /mnt/ess_storage/DN_1/storage/home/khodorchenko/GOTM/datasets/clickhouse_issues

In [671]:
PATH_TO_SAVE = '/mnt/ess_storage/DN_1/storage/home/khodorchenko/GOTM/datasets_small/clickhouse_issues'

In [672]:
all_df = all_df.sample(frac=1).reset_index(drop=True)

In [676]:
! ls -lah /mnt/ess_storage/DN_1/storage/home/khodorchenko/GOTM/datasets_small/clickhouse_issues

total 82M
drwxr-xr-x 3 mkhodorchenko mkhodorchenko 4.0K Dec  4 22:29 .
drwxr-xr-x 9 mkhodorchenko mkhodorchenko 4.0K Dec  4 22:28 ..
drwxr-xr-x 2 mkhodorchenko mkhodorchenko 4.0K Dec  4 22:29 batches
-rw-r--r-- 1 mkhodorchenko mkhodorchenko  12M Dec  4 22:29 cooc_df.txt
-rw-r--r-- 1 mkhodorchenko mkhodorchenko 1.4M Dec  4 22:29 cooc_dictionary.txt
-rw-r--r-- 1 mkhodorchenko mkhodorchenko  12M Dec  4 22:29 cooc_tf.txt
-rw-r--r-- 1 mkhodorchenko mkhodorchenko 7.3M Dec  4 22:28 dataset.csv
-rw-r--r-- 1 mkhodorchenko mkhodorchenko  12M Dec  4 22:29 dataset_processed.csv
-rw-r--r-- 1 mkhodorchenko mkhodorchenko 1.4M Dec  4 22:29 dictionary.txt
-rw-r--r-- 1 mkhodorchenko mkhodorchenko  16M Dec  4 22:29 ppmi_df.txt
-rw-r--r-- 1 mkhodorchenko mkhodorchenko  17M Dec  4 22:29 ppmi_tf.txt
-rw-r--r-- 1 mkhodorchenko mkhodorchenko 3.7M Dec  4 22:29 test_set_data_voc.txt
-rw-r--r-- 1 mkhodorchenko mkhodorchenko 582K Dec  4 22:29 vocab.txt


In [674]:
all_df.to_csv(os.path.join(PATH_TO_SAVE, 'dataset.csv'), index=None)

In [ ]:
! ls /mnt/ess_storage/DN_1/storage/home/khodorchenko/GOTM/datasets/clickhouse_issues